In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from imgaug import augmenters as iaa
import imgaug as ia
import gc
import csv
import keras
import pandas as pd
from keras.models import Sequential
import csv
from keras.layers import GlobalMaxPooling1D, Dense, Conv1D,Conv2D
from keras import regularizers,optimizers
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from keras.models import Model
from keras.layers import (
    Input,
    Dense,
    Flatten,
    merge,
    Lambda
)
from __future__ import print_function

import numpy as np
import warnings
from keras.layers import Input
from keras import layers
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers import ZeroPadding2D
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from keras.preprocessing import image
import keras.backend as K
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.engine.topology import get_source_inputs
from keras.layers.convolutional import (
    Convolution2D,
    MaxPooling2D,
    AveragePooling2D,
    ZeroPadding2D
)
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
import keras.backend as K

from __future__ import print_function
from __future__ import absolute_import

import warnings
import numpy as np

from keras.models import Model
from keras import layers
from keras.layers import Activation
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import AveragePooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalAveragePooling1D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.layer_utils import convert_all_kernels_in_model
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing import image

# Evidently this model breaks Python's default recursion limit
# This is a theano issue
import sys
sys.setrecursionlimit(10000)

from keras.models import Model

In [ ]:
!cp '/content/drive/My Drive/MRNet.zip' '/content/MRNet.zip'

In [ ]:
#!wget http://download.cs.stanford.edu/deep/MRNet-v1.0.zip
!unzip MRNet.zip
data_dir = 'content/MRNet-v1.0'

Archive:  MRNet.zip
warning [MRNet.zip]:  4294967296 extra bytes at beginning or within zipfile
  (attempting to process anyway)
file #1:  bad zipfile offset (local header sig):  4294967296
  (attempting to re-compensate)
   creating: MRNet-v1.0/
  inflating: MRNet-v1.0/valid-abnormal.csv  
   creating: MRNet-v1.0/valid/
   creating: MRNet-v1.0/valid/axial/
  inflating: MRNet-v1.0/valid/axial/1139.npy  
  inflating: MRNet-v1.0/valid/axial/1138.npy  
  inflating: MRNet-v1.0/valid/axial/1249.npy  
  inflating: MRNet-v1.0/valid/axial/1248.npy  
  inflating: MRNet-v1.0/valid/axial/1207.npy  
  inflating: MRNet-v1.0/valid/axial/1213.npy  
  inflating: MRNet-v1.0/valid/axial/1171.npy  
  inflating: MRNet-v1.0/valid/axial/1165.npy  
  inflating: MRNet-v1.0/valid/axial/1159.npy  
  inflating: MRNet-v1.0/valid/axial/1158.npy  
  inflating: MRNet-v1.0/valid/axial/1164.npy  
  inflating: MRNet-v1.0/valid/axial/1170.npy  
  inflating: MRNet-v1.0/valid/axial/1212.npy  
  inflating: MRNet-v1.0/valid

In [ ]:
def augment(image):
  aug=[]
  for s in range(len(image)):
      aug.append(ia.imresize_single_image(image[s], (227, 227)))
  aug=np.array(aug)
  return aug

def clear():
  aug= None
  gc.collect()

def add_rgb(grey_img):
  rgb_img = np.repeat(grey_img[..., np.newaxis], 3, -1)
  return rgb_img

def load_data(path):
  data = []
  for d in sorted(os.listdir(path)):
    if d!='.DS_Store':
      data.append(d)
      print("in iterator ", d)
      clear()
  return data
  
def load_labels(path):
  labels = []
  read = pd.read_csv(path, names=['num', 'hot'])
  labels = list(read['hot'])
  return labels


# AlexNet Feature Extractor Model

In [ ]:
#MRNet - Model
ALEXNet = Sequential()

# 1st Convolutional Layer
ALEXNet.add(Conv2D(filters=96, input_shape=(227,227,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
ALEXNet.add(Activation('relu'))

# Max Pooling
ALEXNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 2nd Convolutional Layer
ALEXNet.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

# Max Pooling
ALEXNet.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

# 3rd Convolutional Layer
ALEXNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

# 4th Convolutional Layer
ALEXNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

# 5th Convolutional Layer
ALEXNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
ALEXNet.add(Activation('relu'))

average_pool = Sequential()
average_pool.add(layers.AveragePooling2D())
average_pool.add(layers.Flatten())
#average_pool.add(layers.Dense(1, activation='sigmoid'))


MRNet = Sequential([
    ALEXNet, 
    average_pool])


MRNet.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
MRNet.add(Dense(1, activation ='sigmoid'))

sgd = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

ALEXNet.summary()
average_pool.summary()
MRNet.summary()

MRNet.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 23, 23, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 23, 23, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 384)         8

In [ ]:
#Load Data
train_axial_dir='MRNet-v1.0/train/axial'
train_coronal_dir= 'MRNet-v1.0/train/coronal'
train_sagittal_dir= 'MRNet-v1.0/train/sagittal'

train_abnormal_labels= '/content/MRNet-v1.0/train-abnormal.csv'
train_acl_labels= '/content/MRNet-v1.0/train-acl.csv'
train_meniscus_labels= '/content/MRNet-v1.0/train-meniscus.csv'

In [ ]:
print("**************     load axial   **************** ")
train_axial_data = os.listdir(train_axial_dir)

print("**************     load cronal   **************** ")
train_coronal_data = os.listdir(train_coronal_dir)

print("**************     load sagital   **************** ")
train_sagittal_data = os.listdir(train_sagittal_dir)

print("**************  load abnormal labels   **************** ")
train_abnormal_labels = load_labels(train_abnormal_labels)

print("**************  load ACL labels   **************** ")
train_acl_labels = load_labels(train_acl_labels)

print("**************  load Meniscus labels   **************** ")
train_meniscus_labels = load_labels(train_meniscus_labels)

**************     load axial   **************** 
**************     load cronal   **************** 
**************     load sagital   **************** 
**************  load abnormal labels   **************** 
**************  load ACL labels   **************** 
**************  load Meniscus labels   **************** 


In [ ]:

val_axial_dir = 'MRNet-v1.0/valid/axial'
val_coronal_dir = 'MRNet-v1.0/valid/coronal'
val_sagittal_dir = 'MRNet-v1.0/valid/sagittal'

val_abnormal_labels= '/content/MRNet-v1.0/valid-abnormal.csv'
val_acl_labels= '/content/MRNet-v1.0/valid-acl.csv'
val_meniscus_labels= '/content/MRNet-v1.0/valid-meniscus.csv'

val_axial_data = os.listdir(val_axial_dir)
val_coronal_data = os.listdir(val_coronal_dir)
val_sagittal_data =  os.listdir(val_sagittal_dir)


val_abnormal_labels= load_labels(val_abnormal_labels)
val_meniscus_labels= load_labels(val_meniscus_labels)
val_acl_labels= load_labels(val_acl_labels)


In [ ]:
!mv '/content/MRNet-v1.0.zip' 'drive/My Drive/MRNet.zip'

mv: cannot stat '/content/MRNet-v1.0.zip': No such file or directory


In [ ]:
#Hyperparameters 
def Average(lst): 
    return sum(lst) / len(lst) 
  
epoch=50
batch_size = 48

epochs=[0]*epoch
for i in range(0,epoch):
  epochs[i]=i

In [ ]:
import random as random

axialt_length = len(train_axial_data)/batch_size + len(train_axial_data)%batch_size
coronalt_length = len(train_coronal_data)/batch_size + len(train_coronal_data)%batch_size
saggitalt_length= len(train_sagittal_data)/batch_size + len(train_sagittal_data)%batch_size


axialv_length = len(val_axial_data)/batch_size + len(val_axial_data)%batch_size
coronalv_length = len(val_coronal_data)/batch_size + len(val_coronal_data)%batch_size
saggitalv_length = len(val_sagittal_data)/batch_size + len(val_sagittal_data)%batch_size


def data_gen(data,label,path,data_length,batch_size=48):
  all_data = list(zip(data,label))
  random.shuffle(all_data)

  imgs = np.zeros(shape=(batch_size,227,227,3))
  label= np.zeros(shape=(batch_size,1))

  j=0

  while(True):
    chunk = all_data[j*batch_size:(j+1)*batch_size]  
    for i,pair in enumerate(chunk):
      if(pair[0] != '.DS_Store'):
        img = np.load(os.path.join(path,pair[0]),allow_pickle=True)
        img = img.astype(np.uint8)
        img = augment(img)
        img = add_rgb(img)
        imgs[i] = img[0,:,:,:]
        label[i] = pair[1]

        #yield (img_aug[0],pair[1])#np.repeat(pair[1],img_aug.shape[0]))
    yield (imgs,label)  
    j += 1
    

    
axial_abnormal = data_gen(train_axial_data , train_abnormal_labels,train_axial_dir,axialt_length)
coronal_abnormal = data_gen(train_coronal_data,train_abnormal_labels,train_coronal_dir,coronalt_length)
sagittal_abnormal= data_gen(train_sagittal_data,train_abnormal_labels,train_sagittal_dir,saggitalt_length)

axial_abnormal_val = data_gen(val_axial_data , val_abnormal_labels,val_axial_dir,axialv_length)
coronal_abnormal_val   = data_gen(val_coronal_data , val_abnormal_labels,val_coronal_dir,coronalv_length)
sagittal_abnormal_val  = data_gen(val_sagittal_data , val_abnormal_labels,val_sagittal_dir,saggitalv_length)

##########################################################
##########################################################

axial_acl = data_gen(train_axial_data , train_acl_labels,train_axial_dir,axialt_length)
coronal_acl = data_gen(train_coronal_data,train_acl_labels,train_coronal_dir,coronalt_length)
sagittal_acl = data_gen(train_sagittal_data,train_acl_labels,train_sagittal_dir,saggitalt_length)

axial_acl_val = data_gen(val_axial_data , val_acl_labels,val_axial_dir,axialv_length)
coronal_acl_val = data_gen(val_coronal_data , val_acl_labels,val_axial_dir,coronalv_length)
sagittal_acl_val = data_gen(val_sagittal_data , val_acl_labels,val_axial_dir,saggitalv_length)

##########################################################
##########################################################

axial_meniscus = data_gen(train_axial_data , train_meniscus_labels,train_axial_dir,axialt_length)
coronal_meniscus = data_gen(train_coronal_data,train_meniscus_labels,train_coronal_dir,coronalt_length)
sagittal_meniscus = data_gen(train_sagittal_data,train_meniscus_labels,train_sagittal_dir,saggitalt_length)

axial_meniscus_val = data_gen(val_axial_data , val_meniscus_labels,val_axial_dir,axialv_length)
coronal_meniscus_val = data_gen(val_coronal_data , val_meniscus_labels,val_axial_dir,coronalv_length)
sagittal_meniscus_val = data_gen(val_sagittal_data , val_meniscus_labels,val_axial_dir,saggitalv_length)


# Abnormal Section

In [ ]:
abnormalAccuracys=[]
axial_abnormal_output = MRNet.fit_generator(axial_abnormal, epochs =epoch,steps_per_epoch=axialt_length,max_queue_size=axialt_length)
axial_abnormal_valacc = MRNet.evaluate_generator(axial_abnormal_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)

abnormalAccuracys.append(axial_abnormal_valacc[1])
#plot_graphs(axial_abnormal_valacc)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/50
51/50 [==============================] - 66s 1s/step - loss: 0.7709 - accuracy: 0.7966
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.1595 - accuracy: 0.9779
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0440 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0193 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 33ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 33ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 33ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 10/50
51/50 [===============

In [ ]:
coronal_abnormal_output = MRNet.fit_generator(coronal_abnormal, epochs =epoch,steps_per_epoch= coronalt_length)
coronal_abnormal_valacc = MRNet.evaluate_generator(coronal_abnormal_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
abnormalAccuracys.append(coronal_abnormal_valacc[1])
#plot_graphs(coronal_abnormal_valacc)

Epoch 1/50
51/50 [==============================] - 53s 1s/step - loss: 0.8784 - accuracy: 0.8068
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0969 - accuracy: 0.9955
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0286 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0065 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 11/5

In [ ]:
sagittal_abnormal_out = MRNet.fit_generator(sagittal_abnormal, epochs =epoch,steps_per_epoch= coronalt_length)
sagittal_abnormal_valacc= MRNet.evaluate(sagittal_abnormal_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)
abnormalAccuracys.append(sagittal_abnormal_valacc[1])
#plot_graphs(sagittal_abnormal_valacc)

Epoch 1/50
51/50 [==============================] - 61s 1s/step - loss: 0.6312 - accuracy: 0.8329
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0709 - accuracy: 1.0000
Epoch 3/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0253 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0141 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 36ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 11/5

# ACL Section

In [ ]:
aclAccuracys=[]
axial_acl_output = MRNet.fit_generator(axial_acl, epochs =epoch,steps_per_epoch= axialt_length)
axial_acl_valacc = MRNet.evaluate_generator(axial_acl_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(axial_acl_valacc[1]) 

Epoch 1/50
51/50 [==============================] - 68s 1s/step - loss: 1.0904 - accuracy: 0.8231
Epoch 2/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0742 - accuracy: 0.9926
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0225 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 11/5

In [ ]:
coronal_acl_out = MRNet.fit_generator(coronal_acl, epochs =epoch,steps_per_epoch= coronalt_length)
coronal_acl_valacc = MRNet.evaluate_generator(coronal_acl_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(coronal_acl_valacc[1])
#plot_graphs(coronal_acl)

Epoch 1/50
51/50 [==============================] - 59s 1s/step - loss: 0.5386 - accuracy: 0.8640
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0224 - accuracy: 1.0000
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0052 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 11/5

In [ ]:
sagittal_acl_out = MRNet.fit_generator(sagittal_acl, epochs =epoch,steps_per_epoch= saggitalt_length)
sagittal_acl_valacc = MRNet.evaluate_generator(sagittal_acl_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(sagittal_acl_valacc[1])
#plot_graphs(sagittal_acl)

Epoch 1/50
51/50 [==============================] - 60s 1s/step - loss: 0.4706 - accuracy: 0.8713
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0198 - accuracy: 1.0000
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0077 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 11/5

# Meniscus Section

In [ ]:
meniscusAccuracys=[]
axial_meniscus_out = MRNet.fit_generator(axial_meniscus, epochs =epoch,steps_per_epoch= axialt_length)
axial_meniscus_valacc = MRNet.evaluate_generator(axial_meniscus_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(axial_meniscus_valacc[1])
#plot_graphs(axial_meniscus)

Epoch 1/50
51/50 [==============================] - 68s 1s/step - loss: 0.8064 - accuracy: 0.7275
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0860 - accuracy: 1.0000
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0260 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0138 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 11/5

In [ ]:
coronal_meniscus_out = MRNet.fit_generator(coronal_meniscus, epochs =epoch,steps_per_epoch= axialt_length)
coronal_meniscus_valacc = MRNet.evaluate_generator(coronal_meniscus_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(coronal_meniscus_valacc[1])
#plot_graphs(coronal_meniscus)

Epoch 1/50
51/50 [==============================] - 59s 1s/step - loss: 0.8833 - accuracy: 0.7786
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0480 - accuracy: 1.0000
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0154 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0086 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0058 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0024 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 11/5

In [ ]:
sagittal_meniscus_out = MRNet.fit_generator(sagittal_meniscus, epochs =epoch,steps_per_epoch= axialt_length)
sagittal_meniscus_valacc = MRNet.evaluate_generator(sagittal_meniscus_val,steps=saggitalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(sagittal_meniscus_valacc[1])

Epoch 1/50
51/50 [==============================] - 61s 1s/step - loss: 0.7330 - accuracy: 0.7631
Epoch 2/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0345 - accuracy: 1.0000
Epoch 3/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0116 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0069 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0049 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0037 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0025 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 2s 34ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 2s 35ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 11/5

In [ ]:
#Final Model Predictions
def combine_accuracies(acc_list):
  weight = float(1/len(acc_list))

  weight_acc = 0
  for acc in acc_list:
    weight_acc += weight * acc
  
  return weight_acc

abnormal_acc = combine_accuracies(abnormalAccuracys)
acl_acc = combine_accuracies(aclAccuracys)
meniscus_acc = combine_accuracies(meniscusAccuracys)

In [ ]:
print('Abnormal Disease Dev-Set Accuracy : ',abnormal_acc * 100.0 ,'%')
print('ACL Disease Dev-Set Accuracy : ',acl_acc *100.0 ,'%')
print('Meniscus Disease Dev-Set Accuracy : ',meniscus_acc *100.0 ,'%')

Abnormal Disease Dev-Set Accuracy :  75.8230447769165 %
ACL Disease Dev-Set Accuracy :  57.20164577166239 %
Meniscus Disease Dev-Set Accuracy :  48.9969124396642 %


# ResNet - Transfer Learning

In [ ]:
Resnet = keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=None, pooling='avg')
for layer in Resnet.layers:
  layer.trainable= False

83689472/83683744 [==============================] - 0s 0us/step


In [ ]:
Extra = Sequential()
Extra.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
Extra.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
Extra.add(Dense(256, activation ='relu',kernel_constraint=keras.constraints.MaxNorm(max_value=2, axis=0)))
Extra.add(Dense(1, activation ='sigmoid'))

MRRes_Abnormal_axial = Sequential([
    Resnet,
    Extra ])

MRRes_Abnormal_coronal = Sequential([
    Resnet,
    Extra ])

MRRes_Abnormal_sagittal = Sequential([
    Resnet,
    Extra ])


MRRes_Acl_axial = Sequential([
    Resnet,
    Extra ])


MRRes_Acl_coronal = Sequential([
    Resnet,
    Extra ])


MRRes_Acl_sagittal = Sequential([
    Resnet,
    Extra ])


MRRes_Meniscus_axial = Sequential([
    Resnet,
    Extra ])

MRRes_Meniscus_coronal = Sequential([
    Resnet,
    Extra ])

MRRes_Meniscus_sagittal = Sequential([
    Resnet,
    Extra ])

sgd = optimizers.SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)

MRRes_Abnormal_axial.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Abnormal_coronal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Abnormal_sagittal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])


MRRes_Acl_axial.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Acl_coronal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Acl_sagittal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])

MRRes_Meniscus_axial.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Meniscus_coronal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])
MRRes_Meniscus_sagittal.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
abnormalAccuracys=[]
axial_abnormal_output = MRRes_Abnormal_axial.fit_generator(axial_abnormal, epochs =50,steps_per_epoch=axialt_length,max_queue_size=axialt_length)
axial_abnormal_valacc = MRRes_Abnormal_axial.evaluate_generator(axial_abnormal_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)

abnormalAccuracys.append(axial_abnormal_valacc[1])

coronal_abnormal_output = MRRes_Abnormal_coronal.fit_generator(coronal_abnormal, epochs =50,steps_per_epoch= coronalt_length)
coronal_abnormal_valacc = MRRes_Abnormal_coronal.evaluate_generator(coronal_abnormal_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
abnormalAccuracys.append(coronal_abnormal_valacc[1])

sagittal_abnormal_out = MRRes_Abnormal_sagittal.fit_generator(sagittal_abnormal, epochs =50,steps_per_epoch= coronalt_length)
sagittal_abnormal_valacc= MRRes_Abnormal_sagittal.evaluate(sagittal_abnormal_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)

abnormalAccuracys.append(sagittal_abnormal_valacc[1])

Epoch 1/50
51/50 [==============================] - 11s 218ms/step - loss: 0.7366 - accuracy: 0.7316
Epoch 2/50
51/50 [==============================] - 12s 226ms/step - loss: 0.2898 - accuracy: 0.9060
Epoch 3/50
51/50 [==============================] - 12s 231ms/step - loss: 0.2234 - accuracy: 0.9338
Epoch 4/50
51/50 [==============================] - 11s 222ms/step - loss: 0.1814 - accuracy: 0.9755
Epoch 5/50
51/50 [==============================] - 11s 217ms/step - loss: 0.1517 - accuracy: 0.9996
Epoch 6/50
51/50 [==============================] - 11s 216ms/step - loss: 0.1286 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 11s 217ms/step - loss: 0.1104 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 11s 218ms/step - loss: 0.0957 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 11s 219ms/step - loss: 0.0836 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 11s 221ms/step - loss: 0.0733 - accura

In [ ]:
abnormalAccuracys

[0.8541666865348816, 0.6458333134651184, 0.7916666865348816]

In [ ]:
aclAccuracys=[]
axial_acl_output = MRRes_Acl_axial.fit_generator(axial_acl, epochs =50,steps_per_epoch= axialt_length)
axial_acl_valacc = MRRes_Acl_axial.evaluate_generator(axial_acl_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(axial_acl_valacc[1]) 

coronal_acl_out = MRRes_Acl_coronal.fit_generator(coronal_acl, epochs =50,steps_per_epoch= coronalt_length)
coronal_acl_valacc = MRRes_Acl_coronal.evaluate_generator(coronal_acl_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(coronal_acl_valacc[1])

sagittal_acl_out = MRRes_Acl_sagittal.fit_generator(sagittal_acl, epochs =50,steps_per_epoch= saggitalt_length)
sagittal_acl_valacc = MRRes_Acl_sagittal.evaluate_generator(sagittal_acl_val,steps=saggitalv_length ,verbose=1,max_queue_size=50,use_multiprocessing=False)
aclAccuracys.append(sagittal_acl_valacc[1])

Epoch 1/50
51/50 [==============================] - 10s 203ms/step - loss: 0.0540 - accuracy: 1.0000
Epoch 2/50
51/50 [==============================] - 10s 203ms/step - loss: 0.0371 - accuracy: 1.0000
Epoch 3/50
51/50 [==============================] - 10s 204ms/step - loss: 0.0286 - accuracy: 1.0000
Epoch 4/50
51/50 [==============================] - 10s 206ms/step - loss: 0.0233 - accuracy: 1.0000
Epoch 5/50
51/50 [==============================] - 11s 208ms/step - loss: 0.0196 - accuracy: 1.0000
Epoch 6/50
51/50 [==============================] - 11s 210ms/step - loss: 0.0170 - accuracy: 1.0000
Epoch 7/50
51/50 [==============================] - 11s 212ms/step - loss: 0.0149 - accuracy: 1.0000
Epoch 8/50
51/50 [==============================] - 11s 213ms/step - loss: 0.0133 - accuracy: 1.0000
Epoch 9/50
51/50 [==============================] - 11s 212ms/step - loss: 0.0120 - accuracy: 1.0000
Epoch 10/50
51/50 [==============================] - 11s 214ms/step - loss: 0.0109 - accura

In [ ]:
aclAccuracys

In [ ]:
meniscusAccuracys=[]
axial_meniscus_output = MRRes_Meniscus.fit_generator(axial_acl, epochs =1,steps_per_epoch= axialt_length)
axial_meniscus_valacc = MRRes_Meniscus.evaluate_generator(axial_acl_val,steps=axialv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(axial_acl_valacc[1]) 

coronal_meniscus_out = MRRes_Meniscus.fit_generator(coronal_meniscus, epochs =1,steps_per_epoch= axialt_length)
coronal_meniscus_valacc = MRRes_Meniscus.evaluate_generator(coronal_meniscus_val,steps=coronalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(coronal_meniscus_valacc[1])

sagittal_meniscus_out = MRRes_Meniscus.fit_generator(sagittal_meniscus, epochs =1,steps_per_epoch= axialt_length)
sagittal_meniscus_valacc = MRRes_Meniscus.evaluate_generator(sagittal_meniscus_val,steps=saggitalv_length,verbose=1,max_queue_size=50,use_multiprocessing=False)
meniscusAccuracys.append(sagittal_meniscus_valacc[1])

In [ ]:
meniscusAccuracys

In [ ]:
abnormal_acc = combine_accuracies(abnormalAccuracys)
acl_acc = combine_accuracies(aclAccuracys)
meniscus_acc = combine_accuracies(meniscusAccuracys)